Explore the DataSet in this Notebook

In [1]:
import pandas as pd
import os
import glob
from groq import Groq
# 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

In [2]:
r2r_dataset = pd.read_json("Dataset/R2R_train_updated.json")
original_dataset = r2r_dataset.copy()
filtered_dataset = r2r_dataset[r2r_dataset['scan'] == "5LpN3gDmAk7"]

In [3]:
import json

client = Groq(
    api_key="",
)

output_dicts = []
count = 0
for i, row in filtered_dataset.iterrows():
    instructions_json = json.dumps(row['instructions'], indent=2)
    if(count%10 == 0):
        print("i = ",count)
    count+=1
    # print("Instructions = ", instructions_json)
    try:
        chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": """You are a navigation goal statement generator given a list of instructions, generate goal statement without the instructions.that responds in JSON schema which should include a list of dictionaries, where each dictionary has keys 'instruction_index' and 'goal statement'. Generate a list of dictionaries like this example."},:
        {
"navigation_goals": [
    {
        "instruction_index": 1,
        "goal_statement": "I want to navigate to the Hallway past the kitchen."
    },
    {
        "instruction_index": 2,
        "goal_statement": "I want to navigate to the Hallway past the clear glass table."
    },
    {
        "instruction_index": 3,
        "goal_statement": "I want to navigate to the Hallway past the Counter on the right."
    }
]
} """},
            {
                "role": "user",
                "content": f"""
            Given the following instructions ,generate a precise goal statement for each instruction, that the robot and generate instructions on his own.Give your answer in a single dictionary, which contains a list dictionaries.Describe the current scene and where to navigate to. The instructions are:
            {instructions_json}
            Note: Donot give me instructions,just a goal statement telling what the above instructions want to acheive, e.g I am here and want to go to there.""",
            }
        ],
        response_format={"type": "json_object"},
        model="llama3-8b-8192",
    )
        # print(chat_completion.choices[0].message.content)
        output_dicts.append(chat_completion.choices[0].message.content)
    except:
        chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": """You are a navigation goal statement generator given a list of instructions, generate goal statement without the instructions.that responds in JSON schema which should include a list of dictionaries, where each dictionary has keys 'instruction_index' and 'goal statement'. Generate a list of dictionaries like this example."},:
        {
"navigation_goals": [
    {
        "instruction_index": 1,
        "goal_statement": "I want to navigate to the Hallway past the kitchen."
    },
    {
        "instruction_index": 2,
        "goal_statement": "I want to navigate to the Hallway past the clear glass table."
    },
    {
        "instruction_index": 3,
        "goal_statement": "I want to navigate to the Hallway past the Counter on the right."
    }
]
} """},
            {
                "role": "user",
                "content": f"""
            Given the following instructions ,generate a precise goal statement for each instruction, that the robot and generate instructions on his own.Give your answer in a single dictionary, which contains a list dictionaries.Describe the current scene and where to navigate to. The instructions are:
            {instructions_json}
            Note: Donot give me instructions,just a goal statement telling what the above instructions want to acheive, e.g I am here and want to go to there.""",
            }
        ],
        response_format={"type": "json_object"},
        model="llama3-8b-8192",
    )
        # print(chat_completion.choices[0].message.content)
        output_dicts.append(chat_completion.choices[0].message.content)

i =  0
i =  10
i =  20
i =  30
i =  40
i =  50
i =  60
i =  70
i =  80


In [4]:
goalList = []
for dic in output_dicts:
    goalList.append([inst['goal_statement'] for inst in json.loads(dic)['navigation_goals']])

# filtered_dataset['goals'] = None
j = 0
for i, row in filtered_dataset.iterrows():
    filtered_dataset.at[i, 'goals'] = goalList[j]
    j+=1
    

In [5]:
# original_dataset['goals'] = None
for i, row in filtered_dataset.iterrows():
    # Find the corresponding row in the original dataset
    original_index = original_dataset.index[original_dataset['path_id'] == row['path_id']].tolist()
    if original_index:
        original_dataset.at[original_index[0], 'goals'] = row['goals']

# Save the updated full dataset to a new JSON file
updated_full_dataset_path = "Dataset/R2R_train_updated.json"
original_dataset.to_json(updated_full_dataset_path, orient="records", lines=True)

print(f"Updated dataset saved to {updated_full_dataset_path}")

Updated dataset saved to Dataset/R2R_train_updated.json


In [7]:
path = r2r_dataset.iloc[0]['path']

print(path)
# Specify the directory where your images are stored
folder_path = 'Dataset/17DRP5sb8fy/matterport_color_images'

# Specify the word the image names should start with
start_word = path[0]

# Construct the search pattern
pattern = os.path.join(folder_path, f'{start_word}*')

# Get a list of all image filenames that start with the specific word
image_files = glob.glob(pattern)

# Print the filenames (only the names, not the full paths)
image_names = [os.path.basename(image) for image in image_files]
print(image_names[5:8])

['10c252c90fa24ef3b698c6f54d984c5c', '77a1a11978b04e9cbf74914c98578ab8', 'b185432bf33645aca813ac2a961b4140', '5e9f4f8654574e699480e90ecdd150c8', '08c774f20c984008882da2b8547850eb', 'da5fa65c13e643719a20cbb818c9a85d']
['10c252c90fa24ef3b698c6f54d984c5c_i0_5.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_0.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_1.jpg']
